# Borda Count NF-BoT-IoT-v2

In [27]:
from itertools import chain
from pathlib import Path
import json
import pandas as pd

def convert(path):
    f = open(path)
    name_file = Path(f.name).stem

    name_file = name_file.replace('ExtraTreesClassifier','ET').replace('GaussianNB','NB').replace('DecisionTreeClassifier','DT')\
    .replace('RandomForestClassifier','RF').replace('LGBMClassifier','LB').replace('XGBClassifier','XB').replace('X_','').replace('A_mRMR_','')

    data_algo = name_file.split('_')

    final_name = f'{data_algo[0]}_{data_algo[-2:-1][0]}_{data_algo[-1:][0]}'
    final_name = final_name.replace('__X','100')
    final_name = final_name.replace('__','')


    data = json.load(f)
    accuracies = data['mean_test_Accuracy']
    f1_scores = data['mean_test_F1-Score']
    mccs = data['mean_test_MCC']
    kappass = data['mean_test_Kappa']
    params = data['params']
    zipSorted=sorted(zip(accuracies,f1_scores,mccs,kappass, params),key = lambda t: t[0],reverse=True)
    my_dict=params
    my_data=[]
    for accuracy, f1, recall, precision, param in zipSorted:
        score={"Algo":final_name ,"Accuracy":accuracy,"F1-Score":f1,'MCC':recall , 'Kappa':precision }
        my_data_tmp =  dict(chain.from_iterable(d.items() for d in (score, param)))
        my_data.append(my_data_tmp)
    return final_name , pd.DataFrame(data=my_data)

In [28]:
from os import listdir
from os.path import isfile, join
mypath = './result_multi_ton'
my_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
my_result_mean = {}
for f in my_files:
  path_file = join(mypath, f)
  k,v = convert(path_file)
  my_result_mean[k] = v

In [29]:
my_result_mean.keys()

dict_keys(['DT100', 'DT20', 'DT40', 'DT60', 'ET100', 'ET20', 'ET40', 'ET60', 'NB100', 'NB20', 'NB40', 'NB60', 'MLP100', 'MLP20', 'MLP40', 'MLP60', 'RF100', 'RF20', 'RF40', 'RF60', 'XB100', 'XB20', 'XB40', 'XB60'])

In [30]:
key_list = [ 'ET20', 'ET40', 'ET60','ET100',  'RF20', 'RF40', 'RF60', 'RF100', 'XB20', 'XB40', 'XB60', 'XB100']
my_list = []
filtered_dict_ensemble = {key: value for key, value in my_result_mean.items() if key in key_list}
for k,v in filtered_dict_ensemble.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_E{x['n_estimators']}D{x['max_depth']}", axis=1)
    filtered_dict_ensemble[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_ensemble[k])

df_ensemble = pd.concat(my_list)

In [31]:
key_list = ['DT100', 'DT20', 'DT40', 'DT60']
my_list = []

filtered_dict_DT = {key: value for key, value in my_result_mean.items() if key in key_list}

for k,v in filtered_dict_DT.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_D{x['max_depth']}", axis=1)
    filtered_dict_DT[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_DT[k])

df_DT = pd.concat(my_list)

In [32]:
key_list = ['MLP100', 'MLP20', 'MLP40', 'MLP60']

my_list = []

filtered_dict_MLP = {key: value for key, value in my_result_mean.items() if key in key_list}

for k,v in filtered_dict_MLP.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_L{x['hidden_layer_sizes']}", axis=1)
    filtered_dict_MLP[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_MLP[k])

df_MLP = pd.concat(my_list)


In [33]:
key_list = ['NB100', 'NB20', 'NB40', 'NB60']

my_list = []

filtered_dict_NB = {key: value for key, value in my_result_mean.items() if key in key_list}

for k,v in filtered_dict_NB.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_L{x['var_smoothing']}", axis=1)
    filtered_dict_NB[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_NB[k])

df_NB = pd.concat(my_list)


In [34]:

my_list = [df_ensemble,df_DT,df_MLP,df_NB]
df_bot = pd.concat(my_list)


. The first cluster contains three MLP models (MLP20_L56, MLP20_L46, and MLP20_L36), two RF models (RF20_E6D8 and RF20_E4D8), and one DT model (DT20_D12

In [35]:
cluster_1_20_bot = ['MLP20_L56','MLP20_L46','MLP20_L36','RF20_E6D8','RF20_E4D8','DT20_D12']

cluster_1_20_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_20_bot)]


In [36]:
cluster_1_20_bot_df

,Algo,MCC,Kappa
3,RF20_E6D8,0.799169,0.794580
4,RF20_E4D8,0.794190,0.789639
4,DT20_D12,0.849170,0.845887
0,MLP20_L56,0.819375,0.815795
1,MLP20_L46,0.813117,0.809780
2,MLP20_L36,0.799379,0.795356


In [39]:
bot_arduino = pd.read_excel('ton_arduino.xlsx')
bot_arduino

def rename_algo(x):
    my_list = x.split('_')
    return f"{my_list[0]}{my_list[1]}_{my_list[2]}"

bot_arduino['Algo'] = bot_arduino['Algo'].apply(lambda x : rename_algo(x) )

In [40]:
bot_arduino_cluster_1_20 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_20_bot)]

df_bot_all_20 = pd.merge(cluster_1_20_bot_df, bot_arduino_cluster_1_20, on='Algo', how='inner')


df_bot_all_20

,Algo,MCC,Kappa,Latency,SRAM,FM
0,RF20_E6D8,0.799169,0.794580,219.2,19164,512
1,RF20_E4D8,0.794190,0.789639,171.2,15176,512
2,DT20_D12,0.849170,0.845887,48.0,22934,512
3,MLP20_L56,0.819375,0.815795,9.6,6410,526
4,MLP20_L46,0.813117,0.809780,9.6,6410,526
5,MLP20_L36,0.799379,0.795356,9.6,6410,526


## Borda Count

In [41]:
import numpy as np
import pandas as pd

def borda_count(my_data_borda):
    # Define the weights
    weights = {'Kappa': 3, 'MCC': 3, 'Latency': 2, 'SRAM': 1, 'FM': 0.5}

    # Extract the columns to be ranked
    columns = ['Kappa', 'MCC', 'Latency', 'SRAM', 'FM']
    
    # Calculate rank points
    rank_points = my_data_borda[columns].values.argsort(axis=0).argsort(axis=0)

    # Apply weights to the rank points
    weighted_rank_points = np.zeros(rank_points.shape)
    for i, col in enumerate(columns):
        weighted_rank_points[:, i] = rank_points[:, i] * weights[col]
    
    # Sum the weighted rank points
    borda_counts = weighted_rank_points.sum(axis=1)

    # Assign the Borda counts to a new column and sort
    my_data_borda['Borda'] = borda_counts
    sorted_scores = my_data_borda.sort_values(by=['Borda', 'Kappa'], ascending=False)

    return sorted_scores

In [42]:

df_bot_all_20_tmp = df_bot_all_20.copy()
df_bot_all_20_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_20[['Latency', 'SRAM', 'FM']]
df_bot_all_20_bc = borda_count(df_bot_all_20_tmp)
df_bot_all_20_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_20_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_20_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
2,DT20_D12,0.849170,0.845887,48.0,22934,512,36.5
3,MLP20_L56,0.819375,0.815795,9.6,6410,526,33.0
4,MLP20_L46,0.813117,0.809780,9.6,6410,526,30.5
5,MLP20_L36,0.799379,0.795356,9.6,6410,526,28.0
0,RF20_E6D8,0.799169,0.794580,219.2,19164,512,8.5
1,RF20_E4D8,0.794190,0.789639,171.2,15176,512,6.0


## 40 %

The first cluster contains five MLP models (MLP40_L56, MLP40_L46, MLP40_L36, MLP40_L26, and MLP40_L16).

In [43]:
cluster_1_40_bot = ['MLP40_L56','MLP40_L46','MLP40_L36','MLP40_L26','MLP40_L16']

cluster_1_40_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_40_bot)]

In [44]:
bot_arduino_cluster_1_40 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_40_bot)]

df_bot_all_40 = pd.merge(cluster_1_40_bot_df, bot_arduino_cluster_1_40, on='Algo', how='inner')


df_bot_all_40

,Algo,MCC,Kappa,Latency,SRAM,FM
0,MLP40_L56,0.867747,0.866635,9.6,6690,806
1,MLP40_L46,0.861379,0.860327,9.6,6690,806
2,MLP40_L36,0.854443,0.853247,9.6,6690,806
3,MLP40_L26,0.848046,0.847177,9.6,6690,806
4,MLP40_L16,0.839701,0.838326,9.6,6690,806


In [45]:
df_bot_all_40.shape

(5, 6)

In [46]:

df_bot_all_40_tmp = df_bot_all_40.copy()
df_bot_all_40_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_40[['Latency', 'SRAM', 'FM']]
df_bot_all_40_bc = borda_count(df_bot_all_40_tmp)
df_bot_all_40_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_40_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_40_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
0,MLP40_L56,0.867747,0.866635,9.6,6690,806,24.0
1,MLP40_L46,0.861379,0.860327,9.6,6690,806,21.5
2,MLP40_L36,0.854443,0.853247,9.6,6690,806,19.0
3,MLP40_L26,0.848046,0.847177,9.6,6690,806,16.5
4,MLP40_L16,0.839701,0.838326,9.6,6690,806,14.0


## 60 %

The first cluster contains five MLP models (MLP60_L56, MLP60_L46, MLP60_L36, MLP60_L26, and MLP60_L16). 

In [47]:
cluster_1_60_bot = ['MLP60_L56','MLP60_L46','MLP60_L36','MLP60_L26','MLP60_L16']

cluster_1_60_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_60_bot)]

In [48]:
bot_arduino_cluster_1_60 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_60_bot)]

df_bot_all_60 = pd.merge(cluster_1_60_bot_df, bot_arduino_cluster_1_60, on='Algo', how='inner')


df_bot_all_60

,Algo,MCC,Kappa,Latency,SRAM,FM
0,MLP60_L56,0.935341,0.935255,9.6,7010,1126
1,MLP60_L46,0.933865,0.933758,9.6,7010,1126
2,MLP60_L36,0.929772,0.929651,9.6,7010,1126
3,MLP60_L26,0.926333,0.926180,9.6,7010,1126
4,MLP60_L16,0.919506,0.919362,9.6,7010,1126


In [49]:
df_bot_all_60.shape

(5, 6)

In [50]:

df_bot_all_60_tmp = df_bot_all_60.copy()
df_bot_all_60_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_60[['Latency', 'SRAM', 'FM']]
df_bot_all_60_bc = borda_count(df_bot_all_60_tmp)
df_bot_all_60_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_60_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_60_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
0,MLP60_L56,0.935341,0.935255,9.6,7010,1126,24.0
1,MLP60_L46,0.933865,0.933758,9.6,7010,1126,21.5
2,MLP60_L36,0.929772,0.929651,9.6,7010,1126,19.0
3,MLP60_L26,0.926333,0.926180,9.6,7010,1126,16.5
4,MLP60_L16,0.919506,0.919362,9.6,7010,1126,14.0


## 100 %

. The first cluster contains two RF models (RF100_E4D8 and RF100_E2D8)

In [51]:
cluster_1_100_bot = ['RF100_E4D8','RF100_E2D8']
cluster_1_100_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_100_bot)]

In [52]:
bot_arduino_cluster_1_100 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_100_bot)]

df_bot_all_100 = pd.merge(cluster_1_100_bot_df, bot_arduino_cluster_1_100, on='Algo', how='inner')


df_bot_all_100

,Algo,MCC,Kappa,Latency,SRAM,FM
0,RF100_E4D8,0.913199,0.912759,192.0,28406,1452
1,RF100_E2D8,0.910021,0.909705,124.8,16002,1452


In [53]:
df_bot_all_100.shape

(2, 6)

In [54]:

df_bot_all_100_tmp = df_bot_all_100.copy()
df_bot_all_100_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_100[['Latency', 'SRAM', 'FM']]
df_bot_all_100_bc = borda_count(df_bot_all_100_tmp)
df_bot_all_100_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_100_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_100_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
0,RF100_E4D8,0.913199,0.912759,192.0,28406,1452,6.0
1,RF100_E2D8,0.910021,0.909705,124.8,16002,1452,3.5
